In [ ]:
#|default_exp processors

# processors
> Some processors for NBProcessor

In [ ]:
#|export
import ast

from nbdev.read import *
from nbdev.imports import *
from nbdev.process import *
from nbdev.showdoc import *
from nbdev.doclinks import *

from execnb.nbio import *
from execnb.shell import *
from fastcore.imports import *
from fastcore.xtras import *
import sys,yaml

In [ ]:
#|hide
from fastcore.test import *

## Helpers

In [ ]:
#|hide
_test_file = '../tests/docs_test.ipynb'

On this page we'll be using this private helper to process a notebook and return the results, to simplify testing:

In [ ]:
def _run_procs(procs=None, preprocs=None, postprocs=None, return_nb=False, path=_test_file):
    nbp = NBProcessor(path, procs, preprocs=preprocs, postprocs=postprocs)
    nbp.process()
    if return_nb: return nbp.nb
    return '\n'.join([str(cell) for cell in nbp.nb.cells])

## Notebook preprocessors

In [ ]:
#|export
def is_frontmatter(nb):
    "List of raw cells in `nb` that contain frontmatter"
    return _celltyp(nb, 'raw').filter(lambda c: _re_fm.search(c.get('source', '')))

def yml2dict(s:str, rm_fence=True):
    "convert a string that is in a yaml format to a dict"
    if rm_fence: 
        match = _re_fm.search(s.strip())
        if match: s = match.group(1)
    return yaml.safe_load(s)

def _get_frontmatter(nb):
    cell = first(is_frontmatter(nb))
    return cell,(yml2dict(cell.source) if cell else {})

In [ ]:
#|export
def populate_language(nb):
    "Insert cell language indicator based on notebook metadata.  You should to use this before `lang_identify`"
    for cell in nb.cells:
        if cell.cell_type == 'code': cell.metadata.language = nb_lang(nb)

In [ ]:
#| export
def insert_warning(nb):
    "Insert Autogenerated Warning Into Notebook after the first cell."
    content = "<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->"
    nb.cells.insert(1, mk_cell(content, 'markdown'))

This preprocessor inserts a warning in the markdown destination that the file is autogenerated.  This warning is inserted in the second cell so we do not interfere with front matter.

In [ ]:
res = _run_procs(preprocs=[insert_warning])
assert "<!-- WARNING: THIS FILE WAS AUTOGENERATED!" in res

In [ ]:
L('foo', None, 'a').filter(lambda x:x == 1)
_tstre = re.compile('a')

In [ ]:
#|export
_def_types = (ast.FunctionDef,ast.AsyncFunctionDef,ast.ClassDef)
def _def_names(cell, shown):
    return [showdoc_nm(o) for o in concat(cell.parsed_())
            if isinstance(o,_def_types) and o.name not in shown and o.name[0]!='_']

def _get_nm(tree):
    i = tree.value.args[0]
    if hasattr(i, 'id'): val = i.id
    else: val = try_attrs(i.value, 'id', 'func', 'attr')
    return f'{val}.{i.attr}' if isinstance(i, ast.Attribute) else i.id

In [ ]:
#|export
_imps = {ast.Import, ast.ImportFrom}

def _show_docs(trees):
    return [t for t in trees if isinstance(t,ast.Expr) and nested_attr(t, 'value.func.id')=='show_doc']

_show_dirs = {'export','exports','exporti','exec_doc'}

def cell_lang(cell): return nested_attr(cell, 'metadata.language', 'python')

def add_show_docs(nb):
    "Add show_doc cells after exported cells, unless they are already documented"
    def _want(c):
        return c.source and c.cell_type=='code' and ('export' in c.directives_ or 'exports' in c.directives_)

    exports = L(cell for cell in nb.cells if _want(cell))
    trees = L(nb.cells).map(NbCell.parsed_).concat()
    shown_docs = {_get_nm(t) for t in _show_docs(trees)}
    for cell in reversed(exports):
        if cell_lang(cell) != 'python': 
            raise ValueError(f'{cell.metadata.language} cell attempted export:\n{cell.source}')
        for nm in _def_names(cell, shown_docs):
            nb.cells.insert(cell.idx_+1, mk_cell(f'show_doc({nm})'))

In [ ]:
res = _run_procs(preprocs=[populate_language, add_show_docs])
assert "show_doc(some_func)'" in res
assert "show_doc(and_another)'" in res
assert "show_doc(another_func)'" not in res

In [ ]:
#|hide
# this test makes sure @patch works
_nb = _run_procs(preprocs=[populate_language, add_show_docs], return_nb=True, path='../tests/showdoc_test.ipynb')
assert r'show_doc(Foo.a_method)' in L(_nb.cells).attrgot('source')

In [ ]:
#| export
_re_title = re.compile(r'^#\s+(.*)[\n\r]+(?:^>\s+(.*))?', flags=re.MULTILINE)
_re_fm = re.compile(r'^---(.*\S+.*)---', flags=re.DOTALL)
_re_defaultexp = re.compile(r'^\s*#\|\s*default_exp\s+(\S+)', flags=re.MULTILINE)

def _celltyp(nb, cell_type): return L(nb.cells).filter(lambda c: c.cell_type == cell_type)
def _istitle(cell): 
    txt = cell.get('source', '')
    return bool(_re_title.search(txt)) if txt else False

In [ ]:
#|export
def _default_exp(nb):
    "get the default_exp from a notebook"
    code_src = L(nb.cells).filter(lambda x: x.cell_type == 'code').attrgot('source')
    default_exp = first(code_src.filter().map(_re_defaultexp.search).filter())
    return default_exp.group(1) if default_exp else None

In [ ]:
#|hide
_testnb = read_nb('../tests/docs_test.ipynb')
test_eq(_default_exp(_testnb), 'foobar')

In [ ]:
#|hide
_yml = """
---
foo: bar
comments:
  hypothesis: 
    theme: clean
categories: [c1, c2]
---
"""

test_eq(yml2dict(_yml), {'foo': 'bar', 'comments': {'hypothesis': {'theme': 'clean'}}, 'categories': ['c1', 'c2']})

In [ ]:
#|export
def yaml_str(s:str):
    "Create a valid YAML string from `s`"
    if s[0]=='"' and s[-1]=='"': return s
    res = s.replace('\\', '\\\\').replace('"', r'\"')
    return f'"{res}"'

In [ ]:
#|export
def nb_fmdict(nb, remove=True): 
    "Infer the front matter from a notebook's markdown formatting"
    md_cells = _celltyp(nb, 'markdown').filter(_istitle)
    if not md_cells: return {}
    cell = md_cells[0]
    title_match = _re_title.match(cell.source)
    if title_match:
        title,desc=title_match.groups()
        flags = re.findall('^-\s+(.*)', cell.source, flags=re.MULTILINE)
        flags = [s.split(':', 1) for s in flags if ':' in s] if flags else []
        flags = merge({k:v for k,v in flags if k and v}, 
                      {'title':yaml_str(title)}, {'description':yaml_str(desc)} if desc else {})
        if remove: cell['source'] = None
        return yml2dict('\n'.join([f"{k}: {flags[k]}" for k in flags]))
    else: return {}

In [ ]:
_testnb = read_nb('../tests/docs_test.ipynb')
_res = nb_fmdict(_testnb)
test_eq(_res, dict(key1='value1', key2='value2', categories=['c1', 'c2'], title='a title', description='A description'))

In [ ]:
#|hide
_testnb2 = read_nb('../tests/directives.ipynb')
test_eq(nb_fmdict(_testnb2), {})

In [ ]:
#|export
def _replace_fm(d:dict, # dictionary you wish to conditionally change
                k:str,  # key to check 
                val:str,# value to check if d[k] == v
                repl_dict:dict #dictionary that will be used as a replacement 
               ):
    "replace key `k` in dict `d` if d[k] == val with `repl_dict`"
    if str(d.get(k, '')).lower().strip() == str(val.lower()).strip():
        d.pop(k)
        d = merge(d, repl_dict)
    return d

def _fp_alias(d):
    "create aliases for fastpages front matter to match Quarto front matter."
    d = _replace_fm(d, 'search_exclude', 'true', {'search':'false'})
    d = _replace_fm(d, 'hide', 'true', {'draft': 'true'})
    return d

In [ ]:
#|hide
_testnb3 = read_nb('../tests/2020-09-01-fastcore.ipynb')
_fm = _fp_alias(nb_fmdict(_testnb3))
assert 'hide' not in _fm and 'draft' in _fm
assert 'search_exclude' not in _fm and 'search' in _fm
assert 'comments' in _fm

In [ ]:
#|export
def _fp_image(d):
    "Correct path of fastpages images"
    prefix = 'images/copied_from_nb/'
    if d.get('image', '').startswith(prefix): d['image'] = d['image'].replace(prefix, '')
    return d

In [ ]:
#|hide
_testnb3 = read_nb('../tests/2020-09-01-fastcore.ipynb')
_fm = _fp_image(nb_fmdict(_testnb3))
test_eq(_fm['image'], 'fastcore_imgs/td.png')

In [ ]:
#|export
def filter_fm(fmdict:dict):
    "Filter front matter"
    keys = ['title', 'description', 'author', 'image', 'categories', 'output-file', 'aliases', 'search', 'draft', 'comments']
    if not fmdict: return {}
    return filter_keys(fmdict, in_(keys))

In [ ]:
#|export
def construct_fm(fmdict:dict):
    "Construct front matter from a dictionary"
    if not fmdict: return None
    return '---\n'+yaml.dump(fmdict)+'\n---'    

In [ ]:
_testdict = nb_fmdict(read_nb('../tests/docs_test.ipynb'))
_res = construct_fm(filter_fm(_testdict))
test_eq(len(_res.splitlines()), 8)
print(_res)

---
categories:
- c1
- c2
description: A description
title: a title

---


In [ ]:
#|export
def insert_frontmatter(nb, fm_dict:dict):
    "Add frontmatter into notebook based on `filter_keys` that exist in `fmdict`."
    fm = construct_fm(fm_dict)
    if fm: nb.cells.insert(0, NbCell(0, dict(cell_type='raw', metadata={}, source=fm, directives_={})))

In [ ]:
#|export
_fp_convert = compose(_fp_alias, _fp_image)

def infer_frontmatter(nb):
    "Insert front matter if it doesn't exist automatically from nbdev styled markdown."
    raw_fm_cell,raw_fm = _get_frontmatter(nb)
    _exp = _default_exp(nb)
    _fmdict = merge(_fp_convert(filter_fm(nb_fmdict(nb))), {'output-file': _exp+'.html'} if _exp else {}, raw_fm)
    if not _fmdict: return
    if raw_fm: raw_fm_cell['source'] = None
    insert_frontmatter(nb, fm_dict=_fmdict)
    nb.frontmatter_=_fmdict

In [ ]:
_raw_res = _run_procs()
_res = _run_procs(preprocs=infer_frontmatter)
assert '# a title' in _raw_res and '# a title' not in _res
assert r'description: A description\n' in _res
assert r'categories:\n- c1\n- c2\n' in _res
assert r'output-file: foobar.html\n' in _res

If you already have front matter in a raw cell that will take precedence over markdown style frontmatter.  For example, this notebook has the the following front matter defined in a raw cell:

In [ ]:
#|echo: false
_res = _run_procs(path='../tests/2020-09-01-fastcore.ipynb', return_nb=True)
print(_res['cells'][0].source)

---
title: fastcore my favorite python library
---


...as well as a Markdown front matter shortcuts like this:

In [ ]:
#|echo: false
print(_res['cells'][1].source)

# "fastcore: An Underrated Python Library"

> A unique python library that extends the python programming language and provides utilities that enhance productivity.
- author: "<a href='https://twitter.com/HamelHusain'>Hamel Husain</a>"
- toc: false
- image: images/copied_from_nb/fastcore_imgs/td.png
- comments: true
- search_exclude: true
- hide: true
- categories: [fastcore, fastai]
- permalink: /fastcore/
- badges: true


When this notebook is processed, the front matter looks like this, as you can see the `title` from the raw front matter takes precedence:

In [ ]:
_res = _run_procs(preprocs=infer_frontmatter, path='../tests/2020-09-01-fastcore.ipynb', return_nb=True)
print(_res['cells'][0].source)

---
author: <a href='https://twitter.com/HamelHusain'>Hamel Husain</a>
categories:
- fastcore
- fastai
comments: true
description: A unique python library that extends the python programming language
  and provides utilities that enhance productivity.
image: fastcore_imgs/td.png
title: fastcore my favorite python library

---


Additionally, as shown above, front matter from fastpages is automatically aliased to conform to Quarto including modifying the image path, which minimizes burden on folks migrating from fastpages.

## Cell processors

In [ ]:
#|export
def nbflags_(nbp, cell, *args):
    "Store flags marked with `nbflags`"
    nbp.nb._nbflags = args

In [ ]:
nbp = NBProcessor('../tests/01_everything.ipynb', nbflags_)
nbp.process()
test_eq(nbp.nb._nbflags, ('foobar',))

In [ ]:
#|export
def add_links(cell):
    "Add links to markdown cells"
    nl = NbdevLookup()
    if cell.cell_type == 'markdown': cell.source = nl.linkify(cell.source)
    for o in cell.get('outputs', []):
        if hasattr(o, 'data') and hasattr(o['data'], 'text/markdown'):
            o.data['text/markdown'] = [nl.link_line(s) for s in o.data['text/markdown']]

In [ ]:
res = _run_procs(add_links)
assert "[`numpy.array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html#numpy.array)" in res
assert "[`ModuleMaker`](https://nbdev.fast.ai/maker.html#modulemaker) but not a link to `foobar`." in res
assert "A link in a docstring: [`ModuleMaker`](https://nbdev.fast.ai/maker.html#modulemaker)." in res
assert "And not a link to <code>dict2nb</code>." in res

Gets rid of colors that are streamed from standard out, which can interfere with static site generators:

In [ ]:
#|export
_re_ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

def strip_ansi(cell):
    "Strip Ansi Characters."
    for outp in cell.get('outputs', []):
        if outp.get('name')=='stdout': outp['text'] = [_re_ansi_escape.sub('', o) for o in outp.text]

In [ ]:
res = _run_procs(strip_ansi)
assert not _re_ansi_escape.findall(res)

In [ ]:
#|export
def strip_hidden_metadata(cell):
    '''Strips "hidden" metadata property from code cells so it doesn't interfere with docs rendering'''
    if cell.cell_type == 'code' and 'metadata' in cell: cell.metadata.pop('hidden',None)

In [ ]:
#|export
def hide_(nbp, cell):
    "Hide cell from output"
    del(cell['source'])

In [ ]:
res = _run_procs(hide_)
assert 'you will not be able to see this cell at all either' not in res

In [ ]:
#|export
def _re_hideline(lang=None): return re.compile(fr'{langs[lang]}\|\s*hide_line\s*$', re.MULTILINE)

def hide_line(cell):
    "Hide lines of code in code cells with the directive `hide_line` at the end of a line of code"
    lang = cell_lang(cell)
    if cell.cell_type == 'code' and _re_hideline(lang).search(cell.source):
        cell.source = '\n'.join([c for c in cell.source.splitlines() if not _re_hideline(lang).search(c)])

In [ ]:
res = _run_procs(hide_line)
assert r"def show():\n    a = 2\n    b = 3" not in res
assert r"def show():\n    a = 2"                in res

In [ ]:
#|hide
# integration test with hide_line
_nb = _run_procs(hide_line, preprocs=[populate_language], path='../tests/APL.ipynb')
assert 'hide_line' not in _nb

In [ ]:
#|export
def filter_stream_(nbp, cell, *words):
    "Remove output lines containing any of `words` in `cell` stream output"
    if not words: return
    for outp in cell.get('outputs', []):
        if outp.output_type == 'stream':
            outp['text'] = [l for l in outp.text if not re.search('|'.join(words), l)]

In [ ]:
res = _run_procs(filter_stream_)
exp=r"'A line\n', 'Another line.\n'"
assert exp in res

In [ ]:
#|export
_magics_pattern = re.compile(r'^\s*(%%|%).*', re.MULTILINE)

def clean_magics(cell):
    "A preprocessor to remove cell magic commands"
    if cell.cell_type == 'code': cell.source = _magics_pattern.sub('', cell.source).strip()

In [ ]:
res = _run_procs(clean_magics)
assert "%%" not in res

In [ ]:
#|export
_langs = 'bash|html|javascript|js|latex|markdown|perl|ruby|sh|svg'
_lang_pattern = re.compile(rf'^\s*%%\s*({_langs})\s*$', flags=re.MULTILINE)

def lang_identify(cell):
    "A preprocessor to identify bash/js/etc cells and mark them appropriately"
    if cell.cell_type == 'code':
        lang = _lang_pattern.findall(cell.source)
        if lang: cell.metadata.language = lang[0]

When we issue a shell command in a notebook with `!`, we need to change the code-fence from `python` to `bash` and remove the `!`:

In [ ]:
res = _run_procs(lang_identify)
assert "'language': 'bash'" in res

In [ ]:
#|export
_re_hdr_dash = re.compile(r'^#+\s+.*\s+-\s*$', re.MULTILINE)

def rm_header_dash(cell):
    "Remove headings that end with a dash -"
    if cell.source:
        src = cell.source.strip()
        if cell.cell_type == 'markdown' and src.startswith('#') and src.endswith(' -'): del(cell['source'])

In [ ]:
res = _run_procs(rm_header_dash)
assert 'some words' in res
assert 'A heading to Hide' not in res
assert 'Yet another heading to hide' not in res

In [ ]:
#|export
_hide_dirs = {'export','exporti', 'exports', 'hide','default_exp'}

def rm_export(cell):
    "Remove cells that are exported or hidden"
    if cell.directives_:
        if cell.directives_.keys() & _hide_dirs: del(cell['source'])

In [ ]:
res = _run_procs(rm_export)
assert 'dontshow' not in res

In [ ]:
#|export
_re_showdoc = re.compile(r'^show_doc', re.MULTILINE)
def _is_showdoc(cell): return cell['cell_type'] == 'code' and _re_showdoc.search(cell.source)

def clean_show_doc(cell):
    "Remove ShowDoc input cells"
    if not _is_showdoc(cell): return
    cell.source = '#|output: asis\n#| echo: false\n' + cell.source

In [ ]:
#|export
def _do_eval(cell):
    if cell_lang(cell) != 'python': return
    trees = cell.parsed_()
    if cell.cell_type != 'code' or not trees: return
    if cell.directives_.get('eval:', [''])[0].lower() == 'false': return
    if cell.directives_.keys() & _show_dirs or filter_ex(trees, risinstance(_imps)): return True
    if _show_docs(trees): return True

In [ ]:
#|export
class exec_show_docs:
    "Execute cells needed for `show_docs` output, including exported cells and imports"
    def __init__(self, nb):
        self.k = CaptureShell()
        if nb_lang(nb) == 'python': self.k.run_cell('from nbdev.showdoc import show_doc')

    def __call__(self, cell):
        fm = getattr(cell.nb, 'frontmatter_', {})
        if str2bool(fm.get('skip_showdoc', False)): return
        if _do_eval(cell): self.k.cell(cell)
        title = fm.get('title', '')
        if self.k.exc: raise Exception(f"Error{' in notebook: '+title if title else ''} in cell {cell.idx_} :\n{cell.source}") from self.k.exc[1]
                           

In [ ]:
res = _run_procs([exec_show_docs])
assert res

## Notebook postprocessors

There are no notebook postprocessors yet.

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()